In [ ]:
import numpy as np 
import pandas as pd 

import os
os.listdir('/kaggle/input/petfinder-adoption-prediction')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/petfinder-adoption-prediction/train/train.csv", index_col="PetID")
train

In [ ]:
test = pd.read_csv("/kaggle/input/petfinder-adoption-prediction/test/test.csv", index_col="PetID")
test

In [ ]:
train.columns

In [ ]:
train.AdoptionSpeed.value_counts().sort_index().rename("freq").to_frame().join(train.AdoptionSpeed.value_counts(1).rename("freq_rel"))

In [ ]:
train.Description[0]

In [ ]:
train.info()

In [ ]:
pd.crosstab(train.VideoAmt, train.AdoptionSpeed)

In [ ]:
pd.crosstab(train.VideoAmt, train.AdoptionSpeed).apply(lambda x: x / x.sum(), axis=1)

In [ ]:
pd.crosstab(train.PhotoAmt, train.AdoptionSpeed)

In [ ]:
pd.crosstab(train.PhotoAmt.clip(0, 10), train.AdoptionSpeed).apply(lambda x: x / x.sum(), axis=1)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score

def trainModel(train, test):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    folds = [(train.iloc[train_idx].index, train.iloc[valid_idx].index)
             for train_idx, valid_idx in kf.split(train.drop("AdoptionSpeed", axis=1), train.AdoptionSpeed)]
    test_probs = pd.DataFrame(np.zeros((test.shape[0], train.AdoptionSpeed.nunique())), index=test.index)
    valid_probs = []
    for i, (train_idx, valid_idx) in enumerate(folds):
        X_train = train.loc[train_idx]
        y_train = X_train.AdoptionSpeed
        X_train = X_train.drop("AdoptionSpeed", axis=1)
        
        X_valid = train.loc[valid_idx]
        y_valid = X_valid.AdoptionSpeed
        X_valid = X_valid.drop("AdoptionSpeed", axis=1)
        
        learner = LGBMClassifier(n_estimators=10000)
        learner.fit(X_train, y_train,  early_stopping_rounds=10, eval_metric="multi_logloss", verbose=100,
                    eval_set=[(X_train, y_train),
                              (X_valid, y_valid)])
        probs = pd.Series(learner.predict_proba(X_valid).argmax(axis=1), index=X_valid.index)
        valid_probs.append(probs)
        res = cohen_kappa_score(y_valid, probs, weights='quadratic')
        print(f"para el fold {i + 1} el resultado fue: {res}")
        test_probs += learner.predict_proba(test)
    valid_probs = pd.concat(valid_probs)
    res = cohen_kappa_score(train.AdoptionSpeed.loc[valid_probs.index], valid_probs, weights='quadratic')
    print(f"Resultado Final: {res}")
    return test_probs.idxmax(axis=1).rename("AdoptionSpeed")

In [ ]:
test_preds = trainModel(train.select_dtypes(exclude="O"), test.select_dtypes(exclude="O"))

In [ ]:
test_preds.to_csv("submission.csv")